In [39]:
from classiq import * 
import math 
import numpy as np

### Encoding


In [40]:
x0 = np.array([1, 1])
M = np.array([[0, 1], [-1, 0]])
k = 10
t = 1

def C(m):
   return np.linalg.norm(x0) * (np.linalg.norm(M) * t) ** m / math.factorial(m)

N = math.sqrt(sum([C(i) for i in range(k)]))


C(10)

1.2471019068545821e-05

In [43]:
# TODO Initialize ancilla registers with the correct quantum states
# NOTE Our b vector is zero, therefore C-V_S2 and C-U_b do not need to be implemented, but could be for future purposes

K = 10 # Accuracy of solution - subject to change
T = math.ceil(math.log2(K+1))
t = 0 # Time of evaluation of output

print("Register sizes:", 1, T, 1)

# TODO Create the gate C-V_S1 and apply it to prepare the ancilla qubits
@qfunc
def prepare_ancilla_regs(ancilla_reg_1: QBit, ancilla_reg_2: QNum):
   Z(ancilla_reg_1)

   # V_s1 = []
   # total = 0

   # for i in range(k+1):
   #    V_s1.append(C(i))
   #    total += C(i)

   # for i in range(k+1):
   #    V_s1[i] /= total
    
   # for i in range(2**T -len(V_s1)): 
   #    V_s1.append(0)

   
   # # V_s1 = CArray(V_s1)
   # print(type(V_s1))

   # prepare_state(out=ancilla_reg_2, bound=0.01, probabilities=V_s1)

# TODO Create the gate C-Ux and apply it to encode |x0> into the work register
@qfunc
def encode_x0(ancilla_reg_1: QBit, work_reg: QBit):
   pass

@qfunc
def create_entanglement(ancilla_reg_2: QNum, work_reg: QBit):
   repeat(
      count=K + 1,
      iteration=lambda i: if_(
         condition=(i % 2 != 0),
         then=lambda: control(
               ctrl=(ancilla_reg_2 == i), stmt_block=lambda: Y(work_reg)
            ),
         ),
      )

@qfunc
def decode():
   pass

@qfunc
def main(ancilla_reg_1: Output[QBit], ancilla_reg_2: Output[QNum], work_reg: Output[QBit]):
   allocate(1, ancilla_reg_1)
   # allocate(T, ancilla_reg_2)
   allocate(1, work_reg) # 2x2 matrix, will change for higher order ODEs if we get to it
   
   V_s1 = []
   total = 0

   for i in range(k+1):
      V_s1.append(C(i))
      total += C(i)

   for i in range(k+1):
      V_s1[i] /= total
    
   for i in range(2**T -len(V_s1)): 
      V_s1.append(0)

   
   # V_s1 = CArray(V_s1)

   prepare_state(out=ancilla_reg_2, bound=0.01, probabilities=V_s1)

   # prepare_ancilla_regs(ancilla_reg_1, ancilla_reg_2)
   # encode_x0(ancilla_reg_1, work_reg)

   # create_entanglement(ancilla_reg_2, work_reg)

Register sizes: 1 4 1


In [44]:
# View circuit on Classiq IDE
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

: 